# Headfixed odor exposure

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import h5py as h5
import os
import scipy.stats

In [ ]:
def cdf(array, ax=None, ignore_nan=True, **kwargs):
    '''Creates cumulative distribution function from array of data points
    Parameters
    ----------
    array : array_like
        Array to plot CDF. If more than one dimension, array will be flattened.
    ax : Axes object, optional
        Axes on which to plot CDF.
    ignore_nan : boolean
        Whether to ignore nan values in `array`.
    **kwargs: keyword arguments
        Keyword arguments for matplotlib.pyplot.plot.

    Returns
    ----------
    ax: axes object containing CDF

    '''
    array = np.array(array).flatten()
    if ignore_nan:
        array = array[~ np.isnan(array)]

    X = np.sort(array)
    Y = (np.arange(len(array), dtype=float) + 1) / len(array)
    if ax:
        ax.plot(X, Y, **kwargs)
    else:
        plt.plot(X, Y, **kwargs)

    return ax

In [ ]:
neural_file = 'odor-neural.csv'
behav_file = 'odor-behav.csv'

to_plot = True
output_dir = './results'
if output_dir and not os.path.isdir(output_dir):
    os.mkdir(output_dir)

# Parameters
pre = -12000
post = 22000
bin_size = 200
xticklabels = np.arange(-10000, 22000, 5000)
xticks = (xticklabels - pre) / bin_size

epoch_ctrl = 'h2o'

idx = pd.IndexSlice

# Import data

In [ ]:
neural_response = pd.read_csv(neural_file, header=[0, 1, 2], index_col=[0, 1, 2])
behav_response = pd.read_csv('odor-behav.csv', header=[0, 1, 2], index_col=[0, 1, 2])

# Create long format dataframe
neural_response_long = neural_response.stack(list(range(neural_response.columns.nlevels))).reset_index().rename({0: 'value'}, axis=1)

# Visualize data

## Behavior

In [ ]:
# Pupil and velocity plots

def behavior_plots(data, vmin=-2, vmax=2, plt_name=None):
    fig, axes = plt.subplots(2, 2, sharex=True, sharey='row', figsize=(12, 5))
    fig_cb, ax_cb = plt.subplots(figsize=(0.25, 4))
    
    for n, (epoch, df) in enumerate(data.groupby(axis=0, level='epoch')):
        df.index = df.index.droplevel('epoch')
        
        C = df.copy()
        C.index = C.index.droplevel('time_epoch')
        sns.heatmap(C.T, ax=axes[0, n], cbar_ax=ax_cb, center=0, vmin=vmin, vmax=vmax)
        im = axes[0, n].collections[0]
        
        Y = df.mean(axis=1)
        E = df.sem(axis=1)
        Y.plot(ax=axes[1, n])
        X = axes[1, n].lines[0].get_xdata()
        axes[1, n].fill_between(X, Y + E, Y - E, alpha=0.2)
        axes[0, n].set_xticks(xticks)
        axes[0, n].set_xticklabels(xticklabels)
        
        if plt_name:
            fig.savefig(plt_name + '.pdf')
            fig.savefig(plt_name + '.png')

for feature, v, fname in zip(['pupil', 'track'], [[-5, 5], [0, 250]], ['fig2f', 'sf1h']):
    ix_col = idx[:, :, feature]
    plt_name = os.path.join(output_dir, fname)
    behavior_plots(behav_response.loc[:, ix_col], vmin=v[0], vmax=v[1], plt_name=plt_name)

## Neural data

In [ ]:
# Neural plots

def activity_heatmap(data, max_z=4, plt_name=None):
    fig, axes  = plt.subplots(ncols=2, sharey='row', sharex=True, figsize=(8, 8))
    fig_cb, ax_cb = plt.subplots(figsize=(0.25, 4))

    ims = []
    for n, (grp, df) in enumerate(data.groupby(axis=0, level='epoch')):
        df.index = df.index.droplevel('epoch')
        
        # Plot heatmap
        C = df.copy()
        C.columns = np.arange(C.shape[1])
        C.index = C.index.droplevel('time_epoch')
        sns.heatmap(C.T, ax=axes[n], cbar_ax=ax_cb, vmin=-max_z, vmax=max_z, yticklabels=C.shape[1] - 1)
        axes[n].set_title(grp)
        ims.append(axes[n].collections[0])
        
        # Remove spines
        sns.despine(ax=axes[n], left=True, bottom=True)
        sns.despine(ax=axes[n], left=True, bottom=True)

    axes[0].set_ylabel('Neurons');
    axes[0].set_ylabel('Average neural response');
    
    fig.tight_layout()
    
    if plt_name:
        fig.savefig(plt_name + '.pdf')
        fig.savefig(plt_name + '.png')

# Sorting
sort_ix = np.argsort(neural_response.loc[('odor', '2 response'), :].mean().mean(level=['subject', 'neuron']).as_matrix())[::-1]

# Plot
data = neural_response.mean(axis=1, level=['subject', 'neuron']).iloc[:, sort_ix]

if to_plot:
    plt_name = os.path.join(output_dir, 'fig2g')
else:
    plt_name = None

activity_heatmap(data, plt_name=plt_name)

In [ ]:
data = neural_response_long.groupby(['epoch', 'time_epoch', 'subject', 'neuron']).mean()['value'].reset_index()

fig, ax = plt.subplots()
time_epochs = ['0 pre', '2 response']
sns.pointplot(data=data, x='time_epoch', y='value', hue='epoch', order=time_epochs, ci=68, ax=ax)

if to_plot:
    plt_name = os.path.join(output_dir, 'fig2i')
    fig.savefig(plt_name + '.pdf')
    fig.savefig(plt_name + '.png')

# Group neurons

## Statistical difference
Define excited/inhibited neurons as those with significant difference between pre and response period to odor using Mann Whitney U test

In [ ]:
thresh_data = neural_response.mean(axis=1, level=['subject', 'neuron'])
n_neurons = thresh_data.shape[1]
alpha = 0.05
p_thresh = alpha / n_neurons  # Bonferroni correction

grouping = pd.DataFrame(
    columns=pd.MultiIndex.from_product([['h2o', 'odor'], ['p', 'sign', 'group']]),
    index=thresh_data.columns,
    dtype=float
)

for ep in ['h2o', 'odor']:
    for n, neuron in enumerate(thresh_data):
        X = thresh_data.loc[(ep, '0 pre'), neuron]
        Y = thresh_data.loc[(ep, '2 response'), neuron]
        if len(np.unique(np.concatenate((X, Y)))) > 1:
            _, grouping.loc[neuron, (ep, 'p')] = scipy.stats.mannwhitneyu(X, Y)
            grouping.loc[neuron, (ep, 'sign')] = np.sign(Y.mean() - X.mean())

        if grouping.loc[neuron, (ep, 'p')] < p_thresh:
            grouping.loc[neuron, (ep, 'group')] = grouping.loc[neuron, (ep, 'sign')] 
        else:
            grouping.loc[neuron, (ep, 'group')] = 0

grouping.loc[:, idx[:, ['sign', 'group']]] = grouping.loc[:, idx[:, ['sign', 'group']]].astype(int)

In [ ]:
X = grouping[('odor', 'group')].value_counts().sort_index()

fig, ax = plt.subplots()
ax.pie(X, labels=X.index, autopct='%1.1f%%')
circle = plt.Circle((0, 0), 0.7071, color='white')
ax.add_artist(circle)
plt.axis('equal');

if to_plot:
    plt_name = os.path.join(output_dir, 'fig2j')
    fig.savefig(plt_name + '.pdf')
    fig.savefig(plt_name + '.png')

In [ ]:
data = thresh_data.copy()
group_key = grouping[('odor', 'group')]
data.columns = pd.MultiIndex.from_tuples(
    [neuron + tuple([group_key[neuron]]) for neuron in thresh_data],
    names=thresh_data.columns.names + ['group']
)
# data = data.stack(['subject', 'neuron', 'group']).reset_index().rename(columns={0: 'sd'})

fig, axes = plt.subplots(ncols=2, sharey=True, figsize=(10, 5))

for ax, (grp, df) in zip(axes.T, data.groupby(axis=0, level='epoch')):
    Y = data.loc[grp].mean(axis=1, level='group').rolling(5, center=True).mean()
    E = data.loc[grp].sem(axis=1, level='group').rolling(5, center=True).mean()
    Y.plot(ax=ax);
    X = ax.lines[0].get_xdata()

    for n in range(Y.shape[1]):
        y = Y.as_matrix()[:, n]
        e = E.as_matrix()[:, n]
        ix = ~(np.isnan(y) + np.isnan(e))
        ax.fill_between(X[ix], y[ix] - e[ix], y[ix] + e[ix], alpha=0.25)

sns.despine(fig=fig)

if to_plot:
    plt_name = os.path.join(output_dir, 'fig2h')
    fig.savefig(plt_name + '.pdf')
    fig.savefig(plt_name + '.png')

# Correlation
Calculate Spearman correlation coefficient between behavioral features and neural activity during pre, response, and post periods.

In [ ]:
features = ['pupil', 'track']

behav_corr_data = behav_response.loc[idx[:, ['0 pre', '2 response', '4 post']], :].unstack('epoch').stack('trial')
neural_corr_data = neural_response.loc[idx[:, ['0 pre', '2 response', '4 post']], :].unstack('epoch').stack('trial')

behav_corr_data = behav_corr_data.groupby(level='time_epoch').apply(lambda x: x.reset_index(drop=True))
neural_corr_data = neural_corr_data.groupby(level='time_epoch').apply(lambda x: x.reset_index(drop=True))

corrs = pd.DataFrame(index=pd.MultiIndex.from_product([features, ['r', 'p']]), columns=neural_corr_data.columns, dtype=float)
for obs_key in neural_corr_data:
    subj, neuron, epoch = obs_key
    obs_neural = neural_corr_data[obs_key]
    for feature in features:
        obs_behav = behav_corr_data[subj, feature, epoch]
        na_ix = obs_behav.isna() | obs_neural.isna()
        corrs.loc[feature, obs_key] = scipy.stats.spearmanr(obs_behav[~na_ix].dropna(), obs_neural[~na_ix].dropna())

group_key = grouping[('odor', 'group')]
corrs.columns = pd.MultiIndex.from_tuples([x + (group_key[x[:2]],) for x in corrs.columns], names=corrs.columns.names + ['group'])
corrs = corrs.T

In [ ]:
# Stats on correlation distributions

neural_corr_stats = pd.DataFrame(
    index=pd.MultiIndex.from_product([['h2o', 'odor'], [-1, 0, 1]], names=['epoch', 'group']),
    columns=pd.MultiIndex.from_product([features, ['t', 'p']], names=['feature', 'stat']),
    dtype=float
)

for odor in ['h2o', 'odor']:
    for feature in features:
        for grp in [-1, 0, 1]:
            neural_corr_stats.loc[(odor, grp), feature] = list(scipy.stats.ttest_1samp(
                np.arctanh(corrs.loc[idx[:, :, odor, grp], (feature, 'r')]),
                    0,
                    nan_policy='omit'
            ))

In [ ]:
data = corrs.T.xs('r', axis=0, level=1)
data.index.name = 'feature'
data = data.stack('epoch')

# group_key = grouping[('odor', 'group')]
# data.columns = pd.MultiIndex.from_tuples(
#     [neuron + tuple([group_key[neuron]]) for neuron in data],
#     names=data.columns.names + ['group']
# )
data = data.stack(['subject', 'neuron', 'group']).reset_index().rename(columns={0: 'r'})

features = ['pupil', 'track']
epochs = ['h2o', 'odor']
g = sns.FacetGrid(data=data, row='feature', col='epoch', hue='group', row_order=features, col_order=epochs)
g.map(cdf, 'r').add_legend()

for subaxes, feature in zip(g.axes, features):
    for ax, epoch in zip(subaxes, epochs):
        ax.set_xlim(-0.75, 0.75)
        ax.set_title(
            '{}\n'.format(ax.get_title()) +
            'inh | t: {:.2e} p: {:.2e}\n'.format(neural_corr_stats.loc[(epoch, -1), (feature, 't')], neural_corr_stats.loc[(epoch, -1), (feature, 'p')]) +
            'nc | t: {:.2e} p: {:.2e}\n'.format(neural_corr_stats.loc[(epoch, 0), (feature, 't')], neural_corr_stats.loc[(epoch, 0), (feature, 'p')]) +
            'exc | t: {:.2e} p: {:.2e}\n'.format(neural_corr_stats.loc[(epoch, 1), (feature, 't')], neural_corr_stats.loc[(epoch, 1), (feature, 'p')])
        )
g.fig.tight_layout(rect=[0, 0, 0.9, 1])

if to_plot:
    plt_name = os.path.join(output_dir, 'fig2lm')
    g.fig.savefig(plt_name + '.pdf')
    g.fig.savefig(plt_name + '.png')